In [203]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pylab as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn import model_selection
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
import pickle

# Multiclass classification models

In this notebook I will create classification models to label the years as per their economic activity. According to bibliography, the world has experience 4 recessions (decrease of GDP) and 4 downturns (low GDP growth) in the last 70 years. I will label them accordingly:

1. Recession-peak: 1975, 1982, 1991, 2009, 2020
2. Recovery: 2 years after each crisis
3. Global-downturn: 1958, 1998, 2001, 2012
4. Expansion: all other years

## IMF dataset

This is the dataset containing global data for 1980-2027.

In [204]:
pd.reset_option('all')
orignal_imf = pd.read_csv(r'C:\Users\luana\Ironhack DA\Unit 9\final_bootcamp_project\csv_files\imf_clean.csv')
orignal_imf

C:\Users\luana\anaconda3\lib\site-packages\pandas\_config\config.py:653: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)
C:\Users\luana\anaconda3\lib\site-packages\pandas\_config\config.py:653: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


,WEO Subject Code,Country,Subject Descriptor,Subject Notes,Units,Scale,Country/Series-specific Notes,1980,1981,1982,...,2019,2020,2021,2022,2023,2024,2025,2026,2027,Estimates Start After
0,NGDP_R,Albania,"Gross domestic product, constant prices",Expressed in billions of national currency uni...,National currency,Billions,Source: IMF Staff Estimates. Official national...,311.514,329.270,338.819,...,837.786,808.617,877.475,912.574,935.388,965.321000,996.211000,1030.080000,1065.110000,2020.0
1,NGDP_RPCH,Albania,"Gross domestic product, constant prices",Annual percentages of constant price GDP are y...,Percent change,None,"See notes for: Gross domestic product, consta...",2.684,5.700,2.900,...,2.088,-3.482,8.516,4.000,2.500,3.200000,3.200000,3.400000,3.400000,2020.0
2,NGDP,Albania,"Gross domestic product, current prices",Expressed in billions of national currency uni...,National currency,Billions,Source: IMF Staff Estimates. Official national...,18.489,19.126,19.698,...,1691.900,1644.080,1889.840,2059.020,2176.660,2287.470000,2403.260000,2530.790000,2664.390000,2020.0
3,NGDPD,Albania,"Gross domestic product, current prices",Values are based upon GDP in national currency...,U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",1.946,2.229,2.296,...,15.399,15.161,18.310,18.256,18.842,19.858000,21.219000,22.528000,23.889000,2020.0
4,PPPGDP,Albania,"Gross domestic product, current prices",These data form the basis for the country weig...,Purchasing power parity; international dollars,Billions,"See notes for: Gross domestic product, curren...",5.759,6.663,7.280,...,41.623,40.658,45.953,51.189,54.338,57.254000,60.184000,63.415000,66.840000,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4040,D_NGDPD,Sub-Saharan Africa,"External debt, total",None,Percent of GDP,None,None,18.911,21.069,21.791,...,41.042,43.811,40.804,39.774,37.853,37.758550,37.758550,37.758550,37.758550,None
4041,DS,Sub-Saharan Africa,"External debt, total debt service",None,U.S. dollars,Billions,None,10.493,10.300,11.741,...,113.329,109.146,127.688,149.875,129.985,42.604425,42.604425,42.604425,42.604425,None
4042,DS_NGDPD,Sub-Saharan Africa,"External debt, total debt service",None,Percent of GDP,None,None,3.350,3.426,3.512,...,6.637,6.380,7.232,7.655,6.163,6.297775,6.297775,6.297775,6.297775,None
4043,DSI,Sub-Saharan Africa,"External debt, total debt service, interest",None,U.S. dollars,Billions,None,3.301,3.541,3.144,...,18.733,18.035,18.286,18.511,20.489,7.451600,7.451600,7.451600,7.451600,None


### Preparing the dataset

In [205]:
# I need to rearrange the dataset to make it suitable for the classification
# I can drop all the categorical columns because they only give information about the indicators
# I will use data until 2020 for the model and then try to label the years 2021-2027 

imf = orignal_imf.select_dtypes(np.number)
imf

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
0,311.514,329.270,338.819,342.546,349.397,344.156,363.428,360.521,355.474,390.310,...,820.653,837.786,808.617,877.475,912.574,935.388,965.321000,996.211000,1030.080000,1065.110000
1,2.684,5.700,2.900,1.100,2.000,-1.500,5.600,-0.800,-1.400,9.800,...,4.019,2.088,-3.482,8.516,4.000,2.500,3.200000,3.200000,3.400000,3.400000
2,18.489,19.126,19.698,19.900,19.645,20.065,20.692,20.531,20.238,22.228,...,1636.730,1691.900,1644.080,1889.840,2059.020,2176.660,2287.470000,2403.260000,2530.790000,2664.390000
3,1.946,2.229,2.296,2.319,2.290,2.339,2.587,2.566,2.530,2.779,...,15.157,15.399,15.161,18.310,18.256,18.842,19.858000,21.219000,22.528000,23.889000
4,5.759,6.663,7.280,7.649,8.083,8.214,8.848,8.994,9.181,10.476,...,40.055,41.623,40.658,45.953,51.189,54.338,57.254000,60.184000,63.415000,66.840000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4040,18.911,21.069,21.791,29.217,38.232,41.154,41.957,38.565,39.362,40.063,...,39.422,41.042,43.811,40.804,39.774,37.853,37.758550,37.758550,37.758550,37.758550
4041,10.493,10.300,11.741,13.934,15.031,16.869,17.096,17.030,17.986,16.911,...,114.102,113.329,109.146,127.688,149.875,129.985,42.604425,42.604425,42.604425,42.604425
4042,3.350,3.426,3.512,5.457,7.972,8.582,7.109,6.116,6.621,5.700,...,7.230,6.637,6.380,7.232,7.655,6.163,6.297775,6.297775,6.297775,6.297775
4043,3.301,3.541,3.144,3.457,3.234,3.887,4.041,4.385,4.650,4.894,...,17.880,18.733,18.035,18.286,18.511,20.489,7.451600,7.451600,7.451600,7.451600


In [206]:
# I need each year to be a row
imf = imf.transpose().reset_index()
imf

,index,0,1,2,3,4,5,6,7,8,...,4035,4036,4037,4038,4039,4040,4041,4042,4043,4044
0,1980,311.514,2.684,18.489,1.946,5.759,5.935,116584.54,5557.56,6919.41,...,0.39000,0.70600,-6.29800,4.9720,60.0180,18.91100,10.493000,3.350000,3.3010,1.040000
1,1981,329.270,5.700,19.126,2.229,6.663,5.809,120786.15,5757.84,7016.15,...,-0.52400,0.19700,-9.89200,-8.5170,66.1260,21.06900,10.300000,3.426000,3.5410,1.055000
2,1982,338.819,2.900,19.698,2.296,7.280,5.814,121689.95,5800.93,7074.57,...,-1.24300,-0.55800,-11.62700,-2.0610,76.5560,21.79100,11.741000,3.512000,3.1440,0.929000
3,1983,342.546,1.100,19.900,2.319,7.649,5.809,120446.72,5741.66,6997.24,...,-0.88400,0.62700,-9.15100,-0.2070,80.0050,29.21700,13.934000,5.457000,3.4570,1.109000
4,1984,349.397,2.000,19.645,2.290,8.083,5.623,120297.85,5734.57,6763.89,...,-0.54900,-1.06200,-1.78900,-0.0870,83.2400,38.23200,15.031000,7.972000,3.2340,1.307000
5,1985,344.156,-1.500,20.065,2.339,8.214,5.830,116082.04,5533.60,6767.92,...,-0.53400,-0.58500,2.56200,0.0820,88.9980,41.15400,16.869000,8.582000,3.8870,1.721000
6,1986,363.428,5.600,20.692,2.587,8.848,5.694,120235.60,5731.60,6845.79,...,-0.35400,0.47700,-1.59500,0.6650,99.5090,41.95700,17.096000,7.109000,4.0410,1.569000
7,1987,360.521,-0.800,20.531,2.566,8.994,5.695,116915.40,5573.33,6657.96,...,-0.85600,-0.38700,-3.68100,3.8690,111.0070,38.56500,17.030000,6.116000,4.3850,1.567000
8,1988,355.474,-1.400,20.238,2.530,9.181,5.693,113124.00,5392.59,6440.37,...,-0.71200,-0.08000,-2.94200,0.1050,110.6700,39.36200,17.986000,6.621000,4.6500,1.542000
9,1989,390.310,9.800,22.228,2.779,10.476,5.695,120916.02,5764.04,6886.27,...,-2.17500,0.35300,-2.02600,2.6340,119.2380,40.06300,16.911000,5.700000,4.8940,1.593000


In [207]:
#Column names from the indicators
columns = ['year']

for i in range(len(orignal_imf)):
    column_name = (orignal_imf['Country'][i]+'_'+orignal_imf['Subject Descriptor'][i])
    column_name = str(column_name).lower().replace(' ','_')
    columns.append(column_name)

len(columns)

4046

In [208]:
imf.columns= columns
imf.head()

,year,"albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_deflator","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_current_prices",...,"sub-saharan_africa_direct_investment,_net","sub-saharan_africa_portfolio_investment,_net","sub-saharan_africa_other_investment,_net",sub-saharan_africa_change_in_reserves,"sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service,_interest","sub-saharan_africa_external_debt,_total_debt_service,_interest"
0,1980,311.514,2.684,18.489,1.946,5.759,5.935,116584.54,5557.56,6919.41,...,0.390,0.706,-6.298,4.972,60.018,18.911,10.493,3.350,3.301,1.040
1,1981,329.270,5.700,19.126,2.229,6.663,5.809,120786.15,5757.84,7016.15,...,-0.524,0.197,-9.892,-8.517,66.126,21.069,10.300,3.426,3.541,1.055
2,1982,338.819,2.900,19.698,2.296,7.280,5.814,121689.95,5800.93,7074.57,...,-1.243,-0.558,-11.627,-2.061,76.556,21.791,11.741,3.512,3.144,0.929
3,1983,342.546,1.100,19.900,2.319,7.649,5.809,120446.72,5741.66,6997.24,...,-0.884,0.627,-9.151,-0.207,80.005,29.217,13.934,5.457,3.457,1.109
4,1984,349.397,2.000,19.645,2.290,8.083,5.623,120297.85,5734.57,6763.89,...,-0.549,-1.062,-1.789,-0.087,83.240,38.232,15.031,7.972,3.234,1.307


In [209]:
imf.tail(10)

,year,"albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_deflator","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_current_prices",...,"sub-saharan_africa_direct_investment,_net","sub-saharan_africa_portfolio_investment,_net","sub-saharan_africa_other_investment,_net",sub-saharan_africa_change_in_reserves,"sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service,_interest","sub-saharan_africa_external_debt,_total_debt_service,_interest"
38,2018,820.653,4.019,1636.73,15.157,40.055,199.442,284678.59,13570.55,567769.71,...,-20.94600,-4.53800,-19.83800,4.6520,646.9380,39.42200,114.102000,7.230000,17.8800,1.021000
39,2019,837.786,2.088,1691.90,15.399,41.623,201.949,290805.66,13862.63,587280.29,...,-30.50500,-18.70200,-11.27500,6.1850,693.7250,41.04200,113.329000,6.637000,18.7330,1.063000
40,2020,808.617,-3.482,1644.08,15.161,40.658,203.320,280984.50,13394.46,571296.57,...,-10.63300,2.39900,-2.45000,-9.5960,717.2530,43.81100,109.146000,6.380000,18.0350,1.083000
41,2021,877.475,8.516,1889.84,18.310,45.953,215.372,305428.18,14559.68,657808.05,...,-69.53400,44.70100,-11.70900,26.5850,741.1080,40.80400,127.688000,7.232000,18.2860,0.975000
42,2022,912.574,4.000,2059.02,18.256,51.189,225.628,318371.83,15176.70,718335.97,...,-31.83700,5.45600,13.07100,-8.9990,766.9290,39.77400,149.875000,7.655000,18.5110,0.909000
43,2023,935.388,2.500,2176.66,18.842,54.338,232.701,327239.88,15599.44,761490.02,...,-40.27200,4.48500,-9.66100,1.0360,803.0300,37.85300,129.985000,6.163000,20.4890,0.938000
44,2024,965.321,3.200,2287.47,19.858,57.254,236.964,338749.82,16148.12,802716.63,...,-13.42365,-4.79135,1.80165,4.4091,246.9388,37.75855,42.604425,6.297775,7.4516,1.191625
45,2025,996.211,3.200,2403.26,21.219,60.184,241.240,350720.75,16718.77,846078.15,...,-13.42365,-4.79135,1.80165,4.4091,246.9388,37.75855,42.604425,6.297775,7.4516,1.191625
46,2026,1030.080,3.400,2530.79,22.528,63.415,245.688,363876.33,17345.89,894000.51,...,-13.42365,-4.79135,1.80165,4.4091,246.9388,37.75855,42.604425,6.297775,7.4516,1.191625
47,2027,1065.110,3.400,2664.39,23.889,66.840,250.153,377613.26,18000.73,944610.78,...,-13.42365,-4.79135,1.80165,4.4091,246.9388,37.75855,42.604425,6.297775,7.4516,1.191625


In [210]:
imf_to_predict = imf[42:48]
imf = imf[:42]
imf.tail()

,year,"albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_deflator","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_current_prices",...,"sub-saharan_africa_direct_investment,_net","sub-saharan_africa_portfolio_investment,_net","sub-saharan_africa_other_investment,_net",sub-saharan_africa_change_in_reserves,"sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service,_interest","sub-saharan_africa_external_debt,_total_debt_service,_interest"
37,2017,788.943,3.802,1550.65,13.053,37.609,196.547,273542.43,13039.70,537640.30,...,-37.375,-23.977,0.493,15.856,602.733,36.549,83.895,5.050,14.005,0.868
38,2018,820.653,4.019,1636.73,15.157,40.055,199.442,284678.59,13570.55,567769.71,...,-20.946,-4.538,-19.838,4.652,646.938,39.422,114.102,7.230,17.880,1.021
39,2019,837.786,2.088,1691.90,15.399,41.623,201.949,290805.66,13862.63,587280.29,...,-30.505,-18.702,-11.275,6.185,693.725,41.042,113.329,6.637,18.733,1.063
40,2020,808.617,-3.482,1644.08,15.161,40.658,203.320,280984.50,13394.46,571296.57,...,-10.633,2.399,-2.450,-9.596,717.253,43.811,109.146,6.380,18.035,1.083
41,2021,877.475,8.516,1889.84,18.310,45.953,215.372,305428.18,14559.68,657808.05,...,-69.534,44.701,-11.709,26.585,741.108,40.804,127.688,7.232,18.286,0.975


In [211]:
# Now I can label the years
imf['year'] = imf['year'].astype(np.number)

crisis_years = [1975, 1982, 1991, 2009, 2020]
recovery_years = [1976, 1977, 1983, 1984, 1992, 1993, 2010, 2011]
downturn_years = [1958, 1998, 2001, 2012]

labels = []
for i in range(len(imf)):
    if imf['year'][i] in crisis_years:
        labels.append('recession')

    elif imf.loc[i]['year'] in recovery_years:
        labels.append('recovery')

    elif imf.loc[i]['year'] in downturn_years:
        labels.append('downturn')
        
    else:
        labels.append('expansion')

imf['year_label'] = labels

C:\Users\luana\AppData\Local\Temp\ipykernel_20392\1806921177.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imf['year'] = imf['year'].astype(np.number)
C:\Users\luana\AppData\Local\Temp\ipykernel_20392\1806921177.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imf['year_label'] = labels


In [212]:
imf[['year', 'year_label']]

,year,year_label
0,1980.0,expansion
1,1981.0,expansion
2,1982.0,recession
3,1983.0,recovery
4,1984.0,recovery
5,1985.0,expansion
6,1986.0,expansion
7,1987.0,expansion
8,1988.0,expansion
9,1989.0,expansion


In [213]:
imf

,year,"albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_deflator","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_current_prices",...,"sub-saharan_africa_portfolio_investment,_net","sub-saharan_africa_other_investment,_net",sub-saharan_africa_change_in_reserves,"sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service,_interest","sub-saharan_africa_external_debt,_total_debt_service,_interest",year_label
0,1980.0,311.514,2.684,18.489,1.946,5.759,5.935,116584.54,5557.56,6919.41,...,0.706,-6.298,4.972,60.018,18.911,10.493,3.350,3.301,1.040,expansion
1,1981.0,329.270,5.700,19.126,2.229,6.663,5.809,120786.15,5757.84,7016.15,...,0.197,-9.892,-8.517,66.126,21.069,10.300,3.426,3.541,1.055,expansion
2,1982.0,338.819,2.900,19.698,2.296,7.280,5.814,121689.95,5800.93,7074.57,...,-0.558,-11.627,-2.061,76.556,21.791,11.741,3.512,3.144,0.929,recession
3,1983.0,342.546,1.100,19.900,2.319,7.649,5.809,120446.72,5741.66,6997.24,...,0.627,-9.151,-0.207,80.005,29.217,13.934,5.457,3.457,1.109,recovery
4,1984.0,349.397,2.000,19.645,2.290,8.083,5.623,120297.85,5734.57,6763.89,...,-1.062,-1.789,-0.087,83.240,38.232,15.031,7.972,3.234,1.307,recovery
5,1985.0,344.156,-1.500,20.065,2.339,8.214,5.830,116082.04,5533.60,6767.92,...,-0.585,2.562,0.082,88.998,41.154,16.869,8.582,3.887,1.721,expansion
6,1986.0,363.428,5.600,20.692,2.587,8.848,5.694,120235.60,5731.60,6845.79,...,0.477,-1.595,0.665,99.509,41.957,17.096,7.109,4.041,1.569,expansion
7,1987.0,360.521,-0.800,20.531,2.566,8.994,5.695,116915.40,5573.33,6657.96,...,-0.387,-3.681,3.869,111.007,38.565,17.030,6.116,4.385,1.567,expansion
8,1988.0,355.474,-1.400,20.238,2.530,9.181,5.693,113124.00,5392.59,6440.37,...,-0.080,-2.942,0.105,110.670,39.362,17.986,6.621,4.650,1.542,expansion
9,1989.0,390.310,9.800,22.228,2.779,10.476,5.695,120916.02,5764.04,6886.27,...,0.353,-2.026,2.634,119.238,40.063,16.911,5.700,4.894,1.593,expansion


### Model

In [222]:
X = imf.drop(['year', 'year_label'],axis=1)
X.shape

(42, 4045)

In [215]:
Y = imf['year_label']

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = pd.DataFrame(encoder.transform(Y))

In [216]:
encoded_Y.value_counts()

1    29
3     6
2     4
0     3
dtype: int64

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.3, shuffle=None)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [218]:
y_test

,0
25,1
16,1
41,1
34,1
14,1
1,1
24,1
26,1
20,1
11,2


In [219]:
y_train

,0
15,1
2,2
29,2
19,1
40,2
27,1
9,1
36,1
22,1
38,1


In [225]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(128, activation='sigmoid', input_shape=(4045,)),
    Dense(64, activation='sigmoid'),      # choosing relu instead of sigmoid, this is somewhat common
    Dense(32, activation='sigmoid'),
#     Dense(16, activation='sigmoid'),
#     Dense(8, activation='sigmoid'), 
    Dense(4, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  to_categorical(y_train), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=20,
  batch_size=6, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

Epoch 1/20
5/5 [==============================] - 1s 4ms/step - loss: 1.1203 - accuracy: 0.6207
Epoch 2/20
5/5 [==============================] - 0s 4ms/step - loss: 1.0446 - accuracy: 0.6207
Epoch 3/20
5/5 [==============================] - 0s 5ms/step - loss: 1.0149 - accuracy: 0.6207
Epoch 4/20
5/5 [==============================] - 0s 4ms/step - loss: 0.9850 - accuracy: 0.6207
Epoch 5/20
5/5 [==============================] - 0s 5ms/step - loss: 0.9625 - accuracy: 0.6207
Epoch 6/20
5/5 [==============================] - 0s 4ms/step - loss: 0.9385 - accuracy: 0.6207
Epoch 7/20
5/5 [==============================] - 0s 4ms/step - loss: 0.9115 - accuracy: 0.6207
Epoch 8/20
5/5 [==============================] - 0s 4ms/step - loss: 0.8849 - accuracy: 0.6207
Epoch 9/20
5/5 [==============================] - 0s 4ms/step - loss: 0.8544 - accuracy: 0.6207
Epoch 10/20
5/5 [==============================] - 0s 5ms/step - loss: 0.8275 - accuracy: 0.6207
Epoch 11/20
5/5 [======================

[0.5591939091682434, 0.692307710647583]

In [226]:
predictions = model.predict(X_test)
print(np.argmax(predictions, axis=1))

predictions

1/1 [==============================] - 0s 109ms/step
[1 1 1 1 3 3 1 1 1 3 1 1 1]


array([[0.05155224, 0.87512183, 0.03038181, 0.04294412],
       [0.08629248, 0.7897549 , 0.04027272, 0.08367993],
       [0.05549894, 0.84445983, 0.05618579, 0.04385544],
       [0.0632197 , 0.8503297 , 0.05064053, 0.03581013],
       [0.09099057, 0.32534903, 0.11636287, 0.46729755],
       [0.10156737, 0.35486552, 0.15753584, 0.38603127],
       [0.05397688, 0.8632169 , 0.02983608, 0.05297014],
       [0.04646463, 0.88156253, 0.03102816, 0.04094474],
       [0.05714579, 0.8667754 , 0.02773906, 0.0483398 ],
       [0.10372864, 0.3388023 , 0.14246902, 0.41500002],
       [0.0630362 , 0.7721534 , 0.04529841, 0.11951201],
       [0.12782349, 0.53951585, 0.13569811, 0.19696252],
       [0.06480353, 0.8504706 , 0.03434389, 0.0503821 ]], dtype=float32)

In [227]:
# The model accuracy is acceptable, however the high accuracy is only achieved because of the imbalanced data.
# The model is predicting every year as expansion
# I will try up- and downsampling

In [228]:
all_data = imf
all_data

,year,"albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_deflator","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_current_prices",...,"sub-saharan_africa_portfolio_investment,_net","sub-saharan_africa_other_investment,_net",sub-saharan_africa_change_in_reserves,"sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service,_interest","sub-saharan_africa_external_debt,_total_debt_service,_interest",year_label
0,1980.0,311.514,2.684,18.489,1.946,5.759,5.935,116584.54,5557.56,6919.41,...,0.706,-6.298,4.972,60.018,18.911,10.493,3.350,3.301,1.040,expansion
1,1981.0,329.270,5.700,19.126,2.229,6.663,5.809,120786.15,5757.84,7016.15,...,0.197,-9.892,-8.517,66.126,21.069,10.300,3.426,3.541,1.055,expansion
2,1982.0,338.819,2.900,19.698,2.296,7.280,5.814,121689.95,5800.93,7074.57,...,-0.558,-11.627,-2.061,76.556,21.791,11.741,3.512,3.144,0.929,recession
3,1983.0,342.546,1.100,19.900,2.319,7.649,5.809,120446.72,5741.66,6997.24,...,0.627,-9.151,-0.207,80.005,29.217,13.934,5.457,3.457,1.109,recovery
4,1984.0,349.397,2.000,19.645,2.290,8.083,5.623,120297.85,5734.57,6763.89,...,-1.062,-1.789,-0.087,83.240,38.232,15.031,7.972,3.234,1.307,recovery
5,1985.0,344.156,-1.500,20.065,2.339,8.214,5.830,116082.04,5533.60,6767.92,...,-0.585,2.562,0.082,88.998,41.154,16.869,8.582,3.887,1.721,expansion
6,1986.0,363.428,5.600,20.692,2.587,8.848,5.694,120235.60,5731.60,6845.79,...,0.477,-1.595,0.665,99.509,41.957,17.096,7.109,4.041,1.569,expansion
7,1987.0,360.521,-0.800,20.531,2.566,8.994,5.695,116915.40,5573.33,6657.96,...,-0.387,-3.681,3.869,111.007,38.565,17.030,6.116,4.385,1.567,expansion
8,1988.0,355.474,-1.400,20.238,2.530,9.181,5.693,113124.00,5392.59,6440.37,...,-0.080,-2.942,0.105,110.670,39.362,17.986,6.621,4.650,1.542,expansion
9,1989.0,390.310,9.800,22.228,2.779,10.476,5.695,120916.02,5764.04,6886.27,...,0.353,-2.026,2.634,119.238,40.063,16.911,5.700,4.894,1.593,expansion


### Downsampling

In [229]:
other_categories = all_data[all_data['year_label']!='expansion']
other_categories['year_label'].value_counts()

recovery     6
recession    4
downturn     3
Name: year_label, dtype: int64

In [230]:
category_expansion = all_data[all_data['year_label']=='expansion']
category_expansion_undersampled = resample(category_expansion, replace=False, n_samples = 6)
category_expansion_undersampled

,year,"albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_constant_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_current_prices","albania_gross_domestic_product,_deflator","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_constant_prices","albania_gross_domestic_product_per_capita,_current_prices",...,"sub-saharan_africa_portfolio_investment,_net","sub-saharan_africa_other_investment,_net",sub-saharan_africa_change_in_reserves,"sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service","sub-saharan_africa_external_debt,_total_debt_service,_interest","sub-saharan_africa_external_debt,_total_debt_service,_interest",year_label
10,1990.0,351.279,-10.000,20.006,2.221,9.782,5.695,106884.08,5095.14,6087.18,...,0.422,-1.089,4.333,173.010,47.315,19.507,5.452,4.410,1.071,expansion
27,2007.0,584.254,5.983,965.528,10.677,23.686,165.258,196717.32,9377.46,325091.66,...,-5.898,0.488,28.006,230.022,20.968,60.796,5.547,5.533,0.526,expansion
35,2015.0,735.657,2.219,1434.310,11.389,33.595,194.969,255374.21,12173.62,497901.56,...,-21.485,8.579,-16.484,476.029,32.607,83.004,6.140,13.256,0.817,expansion
8,1988.0,355.474,-1.400,20.238,2.530,9.181,5.693,113124.00,5392.59,6440.37,...,-0.080,-2.942,0.105,110.670,39.362,17.986,6.621,4.650,1.542,expansion
34,2014.0,719.688,1.774,1395.310,13.246,32.529,193.876,249104.37,11874.74,482954.10,...,-8.625,-10.517,-9.102,445.566,26.221,70.988,4.240,11.040,0.608,expansion
14,1994.0,281.424,9.400,223.571,2.361,8.664,79.443,87738.26,4182.46,69701.86,...,-2.598,-2.177,3.536,200.323,62.662,32.446,9.468,7.562,1.882,expansion


In [231]:
imf_undersampled = pd.concat([other_categories, category_expansion_undersampled], axis=0)
imf_undersampled.shape

(19, 4047)

In [232]:
X_undersampled = imf_undersampled.drop(['year', 'year_label'],axis=1)

Y_undersampled = imf_undersampled['year_label']

encoder = LabelEncoder()
encoder.fit(Y_undersampled)
encoded_Y_undersampled = pd.DataFrame(encoder.transform(Y_undersampled))

X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(X_undersampled, encoded_Y_undersampled,
                                                                            test_size=0.3)


scaler = preprocessing.StandardScaler()
X_train_under = scaler.fit_transform(X_train_under)
X_test_under = scaler.transform(X_test_under)

In [237]:
X_undersampled.shape

(19, 4045)

In [238]:
# Build the model.
model_under = Sequential([ # as far as we know, all networks are sequential
    Dense(128, activation='sigmoid', input_shape=(4045,)),
    Dense(64, activation='sigmoid'),      # choosing relu instead of sigmoid, this is somewhat common
    Dense(4, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model_under.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history_under = model_under.fit(
  X_train_under,
  to_categorical(y_train_under), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=20,
  batch_size=6, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model_under.evaluate(
  X_test_under,
  to_categorical(y_test_under)
)

Epoch 1/20
3/3 [==============================] - 0s 5ms/step - loss: 1.7764 - accuracy: 0.2308
Epoch 2/20
3/3 [==============================] - 0s 6ms/step - loss: 1.2384 - accuracy: 0.4615
Epoch 3/20
3/3 [==============================] - 0s 5ms/step - loss: 1.0403 - accuracy: 0.7692
Epoch 4/20
3/3 [==============================] - 0s 6ms/step - loss: 0.9065 - accuracy: 0.6923
Epoch 5/20
3/3 [==============================] - 0s 6ms/step - loss: 0.8185 - accuracy: 0.6923
Epoch 6/20
3/3 [==============================] - 0s 5ms/step - loss: 0.7334 - accuracy: 0.7692
Epoch 7/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6859 - accuracy: 0.7692
Epoch 8/20
3/3 [==============================] - 0s 4ms/step - loss: 0.6348 - accuracy: 0.8462
Epoch 9/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6053 - accuracy: 0.8462
Epoch 10/20
3/3 [==============================] - 0s 5ms/step - loss: 0.5792 - accuracy: 0.8462
Epoch 11/20
3/3 [======================

[2.0892231464385986, 0.1666666716337204]

In [239]:
predictions = model_under.predict(X_test_under)
print(np.argmax(predictions, axis=1))

predictions

1/1 [==============================] - 0s 64ms/step
[3 3 3 1 3 1]


array([[0.02643569, 0.2696185 , 0.20412655, 0.49981922],
       [0.03921694, 0.1328332 , 0.04001822, 0.78793174],
       [0.02647947, 0.06395682, 0.03171552, 0.8778482 ],
       [0.04423366, 0.7004931 , 0.1406303 , 0.11464298],
       [0.06270721, 0.41904297, 0.03698951, 0.48126033],
       [0.03149334, 0.4239604 , 0.17929727, 0.3652491 ]], dtype=float32)

In [240]:
y_test_under

,0
11,0
3,2
0,2
17,1
13,1
9,3


In [ ]:
# Not much better!

## Upsampling

In [241]:
category_expansion = all_data[all_data['year_label']=='expansion']
category_recession = all_data[all_data['year_label']=='recession']
category_recovery = all_data[all_data['year_label']=='recovery']
category_downturn = all_data[all_data['year_label']=='downturn']

category_recession_oversampled = resample(category_recession,replace=True, n_samples = len(category_expansion))
category_recovery_oversampled = resample(category_recovery,replace=True, n_samples = len(category_expansion))
category_downturn_oversampled = resample(category_downturn,replace=True, n_samples = len(category_expansion))

imf_upsampled= pd.concat([category_expansion,
                          category_recession_oversampled,
                          category_recovery_oversampled,
                          category_downturn_oversampled], axis=0)
imf_upsampled.shape

(116, 4047)

In [242]:
X_updersampled = imf_upsampled.drop(['year', 'year_label'],axis=1)

Y_upsampled = imf_upsampled['year_label']

encoder = LabelEncoder()
encoder.fit(Y_upsampled)
encoded_Y_upsampled = pd.DataFrame(encoder.transform(Y_upsampled))

X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(X_updersampled, encoded_Y_upsampled,
                                                                            test_size=0.3)


scaler = preprocessing.StandardScaler()
X_train_under = scaler.fit_transform(X_train_up)
X_test_under = scaler.transform(X_test_up)

In [246]:
X_updersampled.shape

(116, 4045)

In [243]:
Y_upsampled

0     expansion
1     expansion
5     expansion
6     expansion
7     expansion
        ...    
32     downturn
32     downturn
18     downturn
18     downturn
18     downturn
Name: year_label, Length: 116, dtype: object

In [244]:
# 1: expansion, 2: recession, 3: recovery, 0: downturn

In [245]:
encoded_Y_upsampled

,0
0,1
1,1
2,1
3,1
4,1
...,...
111,0
112,0
113,0
114,0


In [247]:
# Build the model.
model_up = Sequential([ # as far as we know, all networks are sequential
    Dense(128, activation='sigmoid', input_shape=(4045,)),
    Dense(128, activation='sigmoid'),      # choosing relu instead of sigmoid, this is somewhat common
    Dense(32, activation='sigmoid'),
    Dense(16, activation='sigmoid'),
    Dense(8, activation='sigmoid'), 
    Dense(4, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model_up.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history_up = model_up.fit(
  X_train_up,
  to_categorical(y_train_up), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=20,
  batch_size=6, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model_up.evaluate(
  X_test_up,
  to_categorical(y_test_up)
)

Epoch 1/20
14/14 [==============================] - 1s 6ms/step - loss: 1.4775 - accuracy: 0.2716
Epoch 2/20
14/14 [==============================] - 0s 4ms/step - loss: 1.4493 - accuracy: 0.2716
Epoch 3/20
14/14 [==============================] - 0s 4ms/step - loss: 1.4302 - accuracy: 0.2716
Epoch 4/20
14/14 [==============================] - 0s 4ms/step - loss: 1.4184 - accuracy: 0.2716
Epoch 5/20
14/14 [==============================] - 0s 4ms/step - loss: 1.4114 - accuracy: 0.2716
Epoch 6/20
14/14 [==============================] - 0s 4ms/step - loss: 1.4018 - accuracy: 0.2716
Epoch 7/20
14/14 [==============================] - 0s 4ms/step - loss: 1.3981 - accuracy: 0.2716
Epoch 8/20
14/14 [==============================] - 0s 5ms/step - loss: 1.3925 - accuracy: 0.2716
Epoch 9/20
14/14 [==============================] - 0s 4ms/step - loss: 1.3896 - accuracy: 0.2716
Epoch 10/20
14/14 [==============================] - 0s 4ms/step - loss: 1.3883 - accuracy: 0.2716
Epoch 11/20
14/14 [

[1.4027689695358276, 0.20000000298023224]

In [248]:
predictions = model_up.predict(X_test_up)
print(np.argmax(predictions, axis=1))

predictions

2/2 [==============================] - 0s 3ms/step
[1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 3 1]


array([[0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.24196844, 0.27123812, 0.21822278, 0.26857066],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.24196844, 0.27123812, 0.21822278, 0.26857066],
       [0.24352106, 0.26686624, 0.22220705, 0.26740566],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.24196844, 0.27123812, 0.21822278, 0.26857066],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.25177568, 0.27790913, 0.21697238, 0.25334278],
       [0.24196844, 0.27123812, 0.21822278, 0.26857066],
       [0.24196844, 0.27123812, 0.21822278, 0.26857066],
       [0.24548447, 0.2692928 ,

In [250]:
# 1: expansion, 2: recession, 3: recovery, 0: downturn

### Conclusions

None of the models was able to predict any of the recession years. Even with upsampling, almost  every category is being missed.

In [258]:
orignal_imf['Scale'].unique()

array(['Billions', 'None', 'Units', 'Millions'], dtype=object)

# World Bank Data

In [65]:
wb_gdp = pd.read_csv(r'C:\Users\luana\Ironhack DA\Unit 9\final_bootcamp_project\csv_files\wb_gdp_predictions.csv')
wb_gdp

,WLD_gdp_per_capita_constant_us$,WLD_gdp_per_capita_constant_%_growth,ARG_gdp_per_capita_constant_us$,ARG_gdp_per_capita_constant_%_growth,AUS_gdp_per_capita_constant_us$,AUS_gdp_per_capita_constant_%_growth,BRA_gdp_per_capita_constant_us$,CAN_gdp_per_capita_constant_us$,CAN_gdp_per_capita_constant_%_growth,CHN_gdp_per_capita_constant_%_growth,...,KOR_gdp_per_capita_constant_%_growth,TUR_gdp_per_capita_constant_us$,TUR_gdp_per_capita_constant_%_growth,GBR_gdp_per_capita_constant_us$,GBR_gdp_per_capita_constant_%_growth,USA_gdp_per_capita_constant_us$,USA_gdp_per_capita_constant_%_growth,CHN_gdp_per_capita_constant_us$,SAU_gdp_per_capita_%_growth,time
0,3676.575517,2.457848,7637.066652,3.728779,20045.843657,0.464273,2754.246341,13131.899295,1.119387,-26.527644,...,3.809555,2477.795384,-1.268241,15198.153421,1.899262,19253.547329,0.618121,175.023372,1.179570,YR1961
1,3806.403544,3.531222,7451.803394,-2.425843,19815.749592,-1.147839,2851.492888,13847.043915,5.445858,-6.351505,...,0.966642,2553.614058,3.059925,15235.923699,0.248519,20116.235124,4.480669,163.906754,1.179570,YR1962
2,3922.051459,3.038246,6945.957125,-6.788240,20647.485947,4.197350,2786.362446,14318.565243,3.405213,7.622254,...,6.026426,2719.253349,6.486465,15859.449114,4.092469,20701.269947,2.908272,176.400144,1.179570,YR1963
3,4095.196204,4.414647,7532.004475,8.437244,21659.152058,4.899706,2799.641949,14984.486701,4.650755,15.468995,...,6.647770,2800.103587,2.973251,16628.576083,4.849645,21599.818705,4.340549,203.687474,1.179570,YR1964
4,4235.398820,3.423587,8202.112548,8.896809,22509.097404,3.924186,2787.438778,15645.230386,4.409518,14.197889,...,4.624622,2811.423840,0.404280,16876.017357,1.488048,22696.678583,5.078098,232.606796,1.179570,YR1965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,11019.375428,1.531947,12712.970738,-2.994388,58781.046657,0.571103,8622.066599,45109.244486,0.426918,5.575317,...,1.888915,11955.433464,-0.427944,47750.879662,1.099996,60687.232258,1.824124,10155.492945,-1.322726,YR2019
59,10548.904223,-4.269491,11344.405742,-10.765108,58029.515526,-1.278526,8228.774263,42258.691017,-6.319222,1.996619,...,-0.988632,12038.628110,0.695873,43166.155200,-9.601340,58060.313948,-4.328618,10358.259448,-5.639295,YR2020
60,11057.420754,4.820563,12390.808688,9.223956,58780.333061,1.293855,8551.205336,43945.556990,3.991761,8.013345,...,4.204823,13250.658911,10.067848,46209.105175,7.049388,61280.389602,5.546087,11188.302540,1.701948,YR2021
61,11203.033754,2.270873,12361.024482,1.667638,59435.221629,1.724197,8592.317906,44377.128153,2.401821,7.775952,...,5.305532,13699.764379,2.782035,46588.665559,2.332662,61942.943201,2.538427,11916.306804,1.337818,YR2022


In [66]:
wb_gdp ['time'] = wb_gdp ['time'].str.replace('YR', '')

In [67]:
# Dropping 2021,2022 and 2023. I will label them later
wb_gdp ['time'] = wb_gdp ['time'].str.replace('YR', '')
wb_gdp_to_predict = wb_gdp[-2:]
wb_gdp = wb_gdp[:-2]
wb_gdp

,WLD_gdp_per_capita_constant_us$,WLD_gdp_per_capita_constant_%_growth,ARG_gdp_per_capita_constant_us$,ARG_gdp_per_capita_constant_%_growth,AUS_gdp_per_capita_constant_us$,AUS_gdp_per_capita_constant_%_growth,BRA_gdp_per_capita_constant_us$,CAN_gdp_per_capita_constant_us$,CAN_gdp_per_capita_constant_%_growth,CHN_gdp_per_capita_constant_%_growth,...,KOR_gdp_per_capita_constant_%_growth,TUR_gdp_per_capita_constant_us$,TUR_gdp_per_capita_constant_%_growth,GBR_gdp_per_capita_constant_us$,GBR_gdp_per_capita_constant_%_growth,USA_gdp_per_capita_constant_us$,USA_gdp_per_capita_constant_%_growth,CHN_gdp_per_capita_constant_us$,SAU_gdp_per_capita_%_growth,time
0,3676.575517,2.457848,7637.066652,3.728779,20045.843657,0.464273,2754.246341,13131.899295,1.119387,-26.527644,...,3.809555,2477.795384,-1.268241,15198.153421,1.899262,19253.547329,0.618121,175.023372,1.179570,1961
1,3806.403544,3.531222,7451.803394,-2.425843,19815.749592,-1.147839,2851.492888,13847.043915,5.445858,-6.351505,...,0.966642,2553.614058,3.059925,15235.923699,0.248519,20116.235124,4.480669,163.906754,1.179570,1962
2,3922.051459,3.038246,6945.957125,-6.788240,20647.485947,4.197350,2786.362446,14318.565243,3.405213,7.622254,...,6.026426,2719.253349,6.486465,15859.449114,4.092469,20701.269947,2.908272,176.400144,1.179570,1963
3,4095.196204,4.414647,7532.004475,8.437244,21659.152058,4.899706,2799.641949,14984.486701,4.650755,15.468995,...,6.647770,2800.103587,2.973251,16628.576083,4.849645,21599.818705,4.340549,203.687474,1.179570,1964
4,4235.398820,3.423587,8202.112548,8.896809,22509.097404,3.924186,2787.438778,15645.230386,4.409518,14.197889,...,4.624622,2811.423840,0.404280,16876.017357,1.488048,22696.678583,5.078098,232.606796,1.179570,1965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,10625.255930,2.200971,13595.037355,1.757648,57695.571259,0.587161,8498.293906,44325.488337,1.811279,6.301863,...,2.870197,11835.261106,5.794264,46746.641890,1.442929,58215.411628,1.610808,9053.212733,-2.713853,2017
57,10853.111494,2.144471,13105.397163,-3.601610,58447.252651,1.302841,8582.338637,44917.483728,1.335564,6.251701,...,2.462246,12006.815880,1.449523,47231.336887,1.036855,59600.053098,2.378479,9619.192484,0.684349,2018
58,11019.375428,1.531947,12712.970738,-2.994388,58781.046657,0.571103,8622.066599,45109.244486,0.426918,5.575317,...,1.888915,11955.433464,-0.427944,47750.879662,1.099996,60687.232258,1.824124,10155.492945,-1.322726,2019
59,10548.904223,-4.269491,11344.405742,-10.765108,58029.515526,-1.278526,8228.774263,42258.691017,-6.319222,1.996619,...,-0.988632,12038.628110,0.695873,43166.155200,-9.601340,58060.313948,-4.328618,10358.259448,-5.639295,2020


In [68]:
# Label the years

wb_gdp ['time'] = wb_gdp ['time'].str.replace('YR', '')

pre_recession_years = ['1974', '1981','1990', '2008', '2019' ]
crisis_years = ['1975', '1982', '1991', '2009', '2020']
recovery_years = ['1976', '1977', '1983', '1984', '1992', '1993', '2010', '2011']
downturn_years = ['1958', '1998', '2001', '2012']

labels = []
for i in range(len(wb_gdp)):
    if wb_gdp['time'][i] in pre_recession_years:
        labels.append('pre_recession')
    
    elif wb_gdp ['time'][i] in crisis_years:
        labels.append('recession')

    elif wb_gdp['time'][i] in recovery_years:
        labels.append('recovery')

    elif wb_gdp ['time'][i] in downturn_years:
        labels.append('downturn')
        
    else:
        labels.append('expansion')

wb_gdp['year_label'] = labels

C:\Users\luana\AppData\Local\Temp\ipykernel_20392\452098469.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_gdp ['time'] = wb_gdp ['time'].str.replace('YR', '')
C:\Users\luana\AppData\Local\Temp\ipykernel_20392\452098469.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_gdp['year_label'] = labels


In [70]:
X = wb_gdp.drop(['time', 'year_label'],axis=1)

Y = wb_gdp['year_label']

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = pd.DataFrame(encoder.transform(Y))
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.3, shuffle=None)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X.shape)

(61, 39)


In [78]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(64, activation='sigmoid', input_shape=(39,)),
    Dense(32, activation='relu'),      # choosing relu instead of sigmoid, this is somewhat common
#     Dense(32, activation='sigmoid'),
#     Dense(16, activation='sigmoid'),
#     Dense(8, activation='sigmoid'), 
    Dense(5, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  to_categorical(y_train), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=20,
  batch_size=12, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

Epoch 1/20
4/4 [==============================] - 0s 2ms/step - loss: 1.4508 - accuracy: 0.4286
Epoch 2/20
4/4 [==============================] - 0s 2ms/step - loss: 1.2957 - accuracy: 0.6429
Epoch 3/20
4/4 [==============================] - 0s 3ms/step - loss: 1.1876 - accuracy: 0.6429
Epoch 4/20
4/4 [==============================] - 0s 3ms/step - loss: 1.1197 - accuracy: 0.6429
Epoch 5/20
4/4 [==============================] - 0s 2ms/step - loss: 1.0721 - accuracy: 0.6429
Epoch 6/20
4/4 [==============================] - 0s 2ms/step - loss: 1.0325 - accuracy: 0.6429
Epoch 7/20
4/4 [==============================] - 0s 2ms/step - loss: 1.0074 - accuracy: 0.6429
Epoch 8/20
4/4 [==============================] - 0s 3ms/step - loss: 0.9883 - accuracy: 0.6429
Epoch 9/20
4/4 [==============================] - 0s 2ms/step - loss: 0.9717 - accuracy: 0.6429
Epoch 10/20
4/4 [==============================] - 0s 2ms/step - loss: 0.9598 - accuracy: 0.6429
Epoch 11/20
4/4 [======================

[1.051735281944275, 0.6842105388641357]

In [79]:
predictions = model.predict(X_test)
print(np.argmax(predictions, axis=1))

predictions

1/1 [==============================] - 0s 62ms/step
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


array([[0.02232995, 0.8608778 , 0.02413612, 0.00182158, 0.09083451],
       [0.02581779, 0.7837151 , 0.049599  , 0.00390682, 0.13696125],
       [0.07475366, 0.4977029 , 0.11211228, 0.03421671, 0.28121442],
       [0.06392016, 0.7161982 , 0.06335564, 0.01490853, 0.14161748],
       [0.05118024, 0.47273606, 0.1503736 , 0.03815458, 0.28755555],
       [0.06202687, 0.6538112 , 0.07408782, 0.01649163, 0.19358243],
       [0.09202089, 0.74036884, 0.06177533, 0.01329842, 0.09253649],
       [0.1093707 , 0.5176409 , 0.13895665, 0.05398851, 0.18004315],
       [0.02492349, 0.71912646, 0.0468461 , 0.00575557, 0.20334834],
       [0.11319879, 0.60387826, 0.09808969, 0.03289722, 0.15193604],
       [0.0236006 , 0.880494  , 0.0213618 , 0.00182827, 0.07271532],
       [0.04257149, 0.85562557, 0.03416744, 0.00282529, 0.06481031],
       [0.01604337, 0.8255332 , 0.03105077, 0.00255012, 0.12482259],
       [0.07324563, 0.793909  , 0.05877624, 0.00529575, 0.06877336],
       [0.05712935, 0.48534963, 0.

In [ ]:
# Again, it can only predict the expansion years

In [80]:
columns_percentage = ['time', 'year_label']

for column in wb_gdp.columns:
    if '%' in column:
        columns_percentage.append(column)

        
columns_percentage

['time',
 'year_label',
 'WLD_gdp_per_capita_constant_%_growth',
 'ARG_gdp_per_capita_constant_%_growth',
 'AUS_gdp_per_capita_constant_%_growth',
 'CAN_gdp_per_capita_constant_%_growth',
 'CHN_gdp_per_capita_constant_%_growth',
 'FRA_gdp_per_capita_constant_%_growth',
 'DEU_gdp_per_capita_constant_%_growth',
 'IND_gdp_per_capita_constant_%_growth',
 'IDN_gdp_per_capita_constant_%_growth',
 'ITA_gdp_per_capita_constant_%_growth',
 'JPN_gdp_per_capita_constant_%_growth',
 'MEX_gdp_per_capita_constant_%_growth',
 'RUS_gdp_per_capita_constant_%_growth',
 'ZAF_gdp_per_capita_constant_%_growth',
 'KOR_gdp_per_capita_constant_%_growth',
 'TUR_gdp_per_capita_constant_%_growth',
 'GBR_gdp_per_capita_constant_%_growth',
 'USA_gdp_per_capita_constant_%_growth',
 'SAU_gdp_per_capita_%_growth']

In [81]:
gdp_percentage = wb_gdp[columns_percentage]
gdp_percentage

,time,year_label,WLD_gdp_per_capita_constant_%_growth,ARG_gdp_per_capita_constant_%_growth,AUS_gdp_per_capita_constant_%_growth,CAN_gdp_per_capita_constant_%_growth,CHN_gdp_per_capita_constant_%_growth,FRA_gdp_per_capita_constant_%_growth,DEU_gdp_per_capita_constant_%_growth,IND_gdp_per_capita_constant_%_growth,...,ITA_gdp_per_capita_constant_%_growth,JPN_gdp_per_capita_constant_%_growth,MEX_gdp_per_capita_constant_%_growth,RUS_gdp_per_capita_constant_%_growth,ZAF_gdp_per_capita_constant_%_growth,KOR_gdp_per_capita_constant_%_growth,TUR_gdp_per_capita_constant_%_growth,GBR_gdp_per_capita_constant_%_growth,USA_gdp_per_capita_constant_%_growth,SAU_gdp_per_capita_%_growth
0,1961,expansion,2.457848,3.728779,0.464273,1.119387,-26.527644,3.604901,1.734333,1.670482,...,7.486419,11.044073,1.782077,0.925136,1.328117,3.809555,-1.268241,1.899262,0.618121,1.179570
1,1962,expansion,3.531222,-2.425843,-1.147839,5.445858,-6.351505,5.361747,1.734333,0.860770,...,5.487478,7.901714,1.463526,0.925136,3.570428,0.966642,3.059925,0.248519,4.480669,1.179570
2,1963,expansion,3.038246,-6.788240,4.197350,3.405213,7.622254,4.751682,1.734333,3.836414,...,4.842052,7.379857,4.803232,0.925136,4.707754,6.026426,6.486465,4.092469,2.908272,1.179570
3,1964,expansion,4.414647,8.437244,4.899706,4.650755,15.468995,5.248370,1.734333,5.249912,...,1.955533,10.520792,8.481411,0.925136,5.236900,6.647770,2.973251,4.849645,4.340549,1.179570
4,1965,expansion,3.423587,8.896809,3.924186,4.409518,14.197889,3.625023,1.734333,-4.640156,...,2.402046,4.686450,3.816439,0.925136,3.448301,4.624622,0.404280,1.488048,5.078098,1.179570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2017,expansion,2.200971,1.757648,0.587161,1.811279,6.301863,1.994998,2.297206,5.665781,...,1.820334,1.758612,0.931457,1.709386,-0.265301,2.870197,5.794264,1.442929,1.610808,-2.713853
57,2018,expansion,2.144471,-3.601610,1.302841,1.335564,6.251701,1.585879,0.782690,5.354757,...,1.117817,0.711770,1.050314,2.815524,0.113021,2.462246,1.449523,1.036855,2.378479,0.684349
58,2019,pre_recession,1.531947,-2.994388,0.571103,0.426918,5.575317,1.620358,0.827865,2.692090,...,1.665737,-0.100125,-1.269322,2.243980,-1.196091,1.888915,-0.427944,1.099996,1.824124,-1.322726
59,2020,recession,-4.269491,-10.765108,-1.278526,-6.319222,1.996619,-8.034379,-4.647545,-7.515675,...,-8.597873,-4.225555,-9.134044,-2.480729,-7.615876,-0.988632,0.695873,-9.601340,-4.328618,-5.639295


In [82]:
X = gdp_percentage.drop(['time', 'year_label'],axis=1)

Y = gdp_percentage['year_label']

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = pd.DataFrame(encoder.transform(Y))
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.35, shuffle=None)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X.shape)

(61, 19)


In [85]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(128, activation='sigmoid', input_shape=(19,)),
    Dense(128, activation='sigmoid'),      # choosing relu instead of sigmoid, this is somewhat common
#     Dense(32, activation='sigmoid'),
#     Dense(16, activation='sigmoid'),
#     Dense(8, activation='sigmoid'), 
    Dense(5, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  to_categorical(y_train), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=20,
  batch_size=6, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

Epoch 1/20
7/7 [==============================] - 0s 2ms/step - loss: 1.5778 - accuracy: 0.3077
Epoch 2/20
7/7 [==============================] - 0s 2ms/step - loss: 1.3034 - accuracy: 0.5385
Epoch 3/20
7/7 [==============================] - 0s 2ms/step - loss: 1.3241 - accuracy: 0.5385
Epoch 4/20
7/7 [==============================] - 0s 2ms/step - loss: 1.2652 - accuracy: 0.5385
Epoch 5/20
7/7 [==============================] - 0s 2ms/step - loss: 1.2212 - accuracy: 0.5385
Epoch 6/20
7/7 [==============================] - 0s 2ms/step - loss: 1.1952 - accuracy: 0.5897
Epoch 7/20
7/7 [==============================] - 0s 2ms/step - loss: 1.1682 - accuracy: 0.5897
Epoch 8/20
7/7 [==============================] - 0s 2ms/step - loss: 1.1431 - accuracy: 0.5897
Epoch 9/20
7/7 [==============================] - 0s 3ms/step - loss: 1.1242 - accuracy: 0.5897
Epoch 10/20
7/7 [==============================] - 0s 2ms/step - loss: 1.0998 - accuracy: 0.5897
Epoch 11/20
7/7 [======================

[0.6965675950050354, 0.9090909361839294]

In [86]:
predictions = model.predict(X_test)
print(np.argmax(predictions, axis=1))

predictions

1/1 [==============================] - 0s 61ms/step
[1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1]


array([[0.06584857, 0.7444848 , 0.06047594, 0.01750344, 0.11168724],
       [0.03469167, 0.8915385 , 0.02162145, 0.0037205 , 0.04842786],
       [0.0825969 , 0.63770926, 0.0815284 , 0.03765613, 0.1605093 ],
       [0.10078788, 0.6179542 , 0.0921862 , 0.03981062, 0.14926103],
       [0.15107664, 0.45560542, 0.14872003, 0.10047367, 0.1441242 ],
       [0.11870691, 0.46699136, 0.11648406, 0.0765143 , 0.22130333],
       [0.0595787 , 0.7845532 , 0.04640644, 0.01391406, 0.09554759],
       [0.12086118, 0.46841958, 0.11702482, 0.08123288, 0.21246156],
       [0.05921567, 0.8391857 , 0.03660288, 0.00805839, 0.05693728],
       [0.05445022, 0.8185882 , 0.04254254, 0.01027077, 0.07414828],
       [0.05706525, 0.8298099 , 0.04160979, 0.01051725, 0.06099779],
       [0.11190801, 0.19485192, 0.13770527, 0.26128113, 0.29425368],
       [0.07681445, 0.7052567 , 0.06614512, 0.02372521, 0.12805852],
       [0.08838513, 0.59196395, 0.08798251, 0.04298111, 0.18868731],
       [0.14216383, 0.35663995, 0.

In [87]:
y_test

,0
11,1
3,1
10,1
17,1
41,1
...,...
26,1
13,2
24,1
53,1


In [88]:
# Removing the columns in dolar seems to have improved the predictions
# I will upsample again and downsample again


category_expansion = gdp_percentage[gdp_percentage['year_label']=='expansion']
category_pre_recession = gdp_percentage[gdp_percentage['year_label']=='pre_recession']
category_recession = gdp_percentage[gdp_percentage['year_label']=='recession']
category_recovery = gdp_percentage[gdp_percentage['year_label']=='recovery']
category_downturn = gdp_percentage[gdp_percentage['year_label']=='downturn']

category_pre_recession_oversampled = resample(category_pre_recession,replace=True, n_samples = len(category_expansion))
category_recession_oversampled = resample(category_recession,replace=True, n_samples = len(category_expansion))
category_recovery_oversampled = resample(category_recovery,replace=True, n_samples = len(category_expansion))
category_downturn_oversampled = resample(category_downturn,replace=True, n_samples = len(category_expansion))

gdp_percentage_upsampled = pd.concat([category_expansion,
                                      category_pre_recession,
                                      category_recession_oversampled,
                                      category_recovery_oversampled,
                                      category_downturn_oversampled], axis=0)

gdp_percentage_upsampled.shape

(165, 21)

In [89]:
X = gdp_percentage_upsampled.drop(['time', 'year_label'],axis=1)

Y = gdp_percentage_upsampled['year_label']

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = pd.DataFrame(encoder.transform(Y))
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.35, shuffle=None)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X.shape)

(165, 19)


In [91]:
# 1: Expansion, 2: pre_recession , 3:recession, 4:recovery , 0: downturn

In [98]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(128, activation='sigmoid', input_shape=(19,)),
    Dense(128, activation='sigmoid'),      # choosing relu instead of sigmoid, this is somewhat common
#     Dense(32, activation='sigmoid'),
#     Dense(16, activation='sigmoid'),
#     Dense(8, activation='sigmoid'), 
    Dense(5, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  to_categorical(y_train), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=20,
  batch_size=6, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

Epoch 1/20
18/18 [==============================] - 1s 2ms/step - loss: 1.5803 - accuracy: 0.1776 
Epoch 2/20
18/18 [==============================] - 0s 4ms/step - loss: 1.4123 - accuracy: 0.4206
Epoch 3/20
18/18 [==============================] - 0s 2ms/step - loss: 1.3701 - accuracy: 0.4019
Epoch 4/20
18/18 [==============================] - 0s 2ms/step - loss: 1.3062 - accuracy: 0.4299
Epoch 5/20
18/18 [==============================] - 0s 2ms/step - loss: 1.1930 - accuracy: 0.6542
Epoch 6/20
18/18 [==============================] - 0s 2ms/step - loss: 1.1278 - accuracy: 0.6075
Epoch 7/20
18/18 [==============================] - 0s 2ms/step - loss: 1.0595 - accuracy: 0.6168
Epoch 8/20
18/18 [==============================] - 0s 2ms/step - loss: 0.9833 - accuracy: 0.7196
Epoch 9/20
18/18 [==============================] - 0s 2ms/step - loss: 0.9003 - accuracy: 0.7477
Epoch 10/20
18/18 [==============================] - 0s 2ms/step - loss: 0.8391 - accuracy: 0.7944
Epoch 11/20
18/18 

[0.39473795890808105, 0.931034505367279]

In [99]:
predictions = model.predict(X_test)
print(np.argmax(predictions, axis=1))

2/2 [==============================] - 0s 2ms/step
[3 0 0 4 3 1 4 3 4 4 4 4 0 1 4 4 0 4 3 1 3 1 0 4 0 4 3 0 3 3 0 1 3 0 1 4 1
 1 4 4 3 4 1 1 1 0 1 3 3 0 0 0 4 3 4 0 0 3]


In [100]:
predictions_array =np.argmax(predictions, axis=1)
y_test_df = pd.DataFrame(y_test).reset_index(drop=True)
y_test_df.columns= ['y_test']
predictions_df = pd.DataFrame(predictions_array)
predictions_df.columns= ['predicted_y']
predictions_df = pd.concat([predictions_df,y_test_df], axis=1)
pd.set_option('display.max_rows',None)
predictions_df

,predicted_y,y_test
0,3,3
1,0,0
2,0,0
3,4,4
4,3,3
5,1,1
6,4,4
7,3,3
8,4,4
9,4,4


In [101]:
# This model has a much better accuracy! Will be used for predictions
model_upsampled = model 
scaler_upsampled  = scaler

filename = 'model_gdp_upsampled.sav'
pickle.dump(model, open(filename, 'wb'))

filename= 'scaler_gdp_upsample.sav'
pickle.dump(scaler, open(filename, 'wb'))

INFO:tensorflow:Assets written to: C:\Users\luana\AppData\Local\Temp\tmpd2caayv7\assets


In [ ]:
# Just in case I will also downsample
gdp_percentage['year_label'].value_counts()

In [ ]:
category_expansion = gdp_percentage[gdp_percentage['year_label']=='expansion']
other_categories = gdp_percentage[gdp_percentage['year_label']!='expansion']
category_category_expansion_undersampled = resample(category_expansion,replace=True, n_samples = 8)

gdp_percentage_undersampled = pd.concat([other_categories, category_category_expansion_undersampled], axis=0)

gdp_percentage_undersampled.shape

In [ ]:
X = gdp_percentage_undersampled.drop(['time', 'year_label'],axis=1)

Y = gdp_percentage_undersampled['year_label']

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = pd.DataFrame(encoder.transform(Y))
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.15, shuffle=None)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X.shape)

In [ ]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(128, activation='sigmoid', input_shape=(20,)),
    Dense(128, activation='sigmoid'),      # choosing relu instead of sigmoid, this is somewhat common
    Dense(32, activation='sigmoid'),
    Dense(16, activation='sigmoid'),
    Dense(8, activation='sigmoid'), 
    Dense(4, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  to_categorical(y_train), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=20,
  batch_size=6, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

In [ ]:
predictions = model.predict(X_test)
print(np.argmax(predictions, axis=1))

In [ ]:
y_test

In [ ]:
y_train

In [ ]:
# Not great... moving on with the upsampled model

In [ ]:
wb_gdp_to_predict

In [104]:
X = wb_gdp_to_predict.drop('time', axis=1)
X = wb_gdp_to_predict.drop([column for column in wb_gdp_to_predict.columns if '%' not in column], axis=1)
X

,WLD_gdp_per_capita_constant_%_growth,ARG_gdp_per_capita_constant_%_growth,AUS_gdp_per_capita_constant_%_growth,CAN_gdp_per_capita_constant_%_growth,CHN_gdp_per_capita_constant_%_growth,FRA_gdp_per_capita_constant_%_growth,DEU_gdp_per_capita_constant_%_growth,IND_gdp_per_capita_constant_%_growth,IDN_gdp_per_capita_constant_%_growth,ITA_gdp_per_capita_constant_%_growth,JPN_gdp_per_capita_constant_%_growth,MEX_gdp_per_capita_constant_%_growth,RUS_gdp_per_capita_constant_%_growth,ZAF_gdp_per_capita_constant_%_growth,KOR_gdp_per_capita_constant_%_growth,TUR_gdp_per_capita_constant_%_growth,GBR_gdp_per_capita_constant_%_growth,USA_gdp_per_capita_constant_%_growth,SAU_gdp_per_capita_%_growth
61,2.270873,1.667638,1.724197,2.401821,7.775952,3.252823,1.871355,3.517629,2.978407,3.338313,2.29909,2.018963,3.672524,1.907525,5.305532,2.782035,2.332662,2.538427,1.337818
62,1.925467,1.027657,1.807011,2.120703,7.687219,2.359089,1.752003,3.220110,3.093034,2.282223,2.42133,1.656412,2.693074,1.213439,5.707932,2.906371,1.947597,2.071957,1.229318


In [105]:
# load the model from disk

filename = 'model_gdp_upsampled.sav'
loaded_model = pickle.load(open(filename, 'rb'))

filename= 'scaler_gdp_upsample.sav'
loaded_scaler = pickle.load(open(filename, 'rb'))

In [106]:
X_scaled = loaded_scaler.transform(X)

predictions = loaded_model.predict(X_scaled)
print(np.argmax(predictions, axis=1))

1/1 [==============================] - 0s 58ms/step
[1 1]


In [ ]:
# 1: Expansion, 2: pre_recession , 3:recession, 4:recovery , 0: downturn

# Conclusion:

#### This model has predicted expansion for 2022 and 2023

# World Bank: data from 1981

In [52]:
pd.set_option('display.max_rows', 10)
wb_1981 = pd.read_csv(r'C:\Users\luana\Ironhack DA\Unit 9\final_bootcamp_project\csv_files\wb_from_1981_predictions.csv')
wb_1981

,time,WLD_gdp_per_capita_constant_us$,WLD_gdp_per_capita_constant_%_growth,WLD_consumer_price_index_%,WLD_unemployment_%_of_total_labor_force,ARG_gdp_per_capita_constant_us$,ARG_gdp_per_capita_constant_%_growth,ARG_unemployment_%_of_total_labor_force,ARG_current_account_balance_us$,ARG_industry_value_added_%_growth,...,USA_unemployment_%_of_total_labor_force,USA_government_debt_total_local_currency,USA_government_debt_total_%_of_gdp,USA_government_expense_local_currency,USA_government_expense_%_gdp,USA_current_account_balance_us$,USA_industry_value_added_us$,USA_net_trade_goods_services_us$,CHN_gdp_per_capita_constant_us$,SAU_gdp_per_capita_%_growth
0,YR1982,5849.893784,-1.381191,10.240268,5.697886,9407.874641,-2.307766,10.977871,-2.353000e+09,-3.209381,...,5.913032,9.779907e+12,67.388391,7.511200e+11,22.463140,-1.160700e+10,22.463140,-2.353700e+10,480.310474,-25.614130
1,YR1983,5899.646053,0.850482,8.771147,5.697886,9660.192130,2.681982,10.977871,-2.436000e+09,5.377688,...,5.913032,9.779907e+12,67.388391,8.286600e+11,22.802733,-4.422200e+10,22.802733,-5.713500e+10,524.408437,-21.161576
2,YR1984,6069.455639,2.878301,8.116398,5.697886,9655.183649,-0.051847,10.977871,-2.495000e+09,0.674286,...,5.913032,9.779907e+12,67.388391,8.668900e+11,21.470359,-9.900800e+10,21.470359,-1.082770e+11,596.200056,-10.171216
3,YR1985,6185.809777,1.917044,6.856812,5.697886,9009.001729,-6.692591,10.977871,-9.520000e+08,-8.651012,...,5.913032,9.779907e+12,67.388391,9.621600e+11,22.174802,-1.244550e+11,22.174802,-1.211020e+11,667.127364,-14.608777
4,YR1986,6287.644923,1.646270,5.822667,5.697886,9412.994813,4.484327,10.977871,-2.859000e+09,8.630264,...,5.913032,9.779907e+12,67.388391,1.018080e+12,22.230612,-1.471740e+11,22.230612,-1.385270e+11,716.104075,11.285443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,YR2019,11019.375428,1.531947,2.186902,5.356915,12712.970738,-2.994388,9.840000,-3.709999e+09,-4.790141,...,3.670000,2.157670e+13,100.955077,4.860451e+12,22.741536,-4.721450e+11,22.741536,-5.763380e+11,10155.492945,-1.322726
38,YR2020,10548.904223,-4.269491,1.920968,6.573234,11344.405742,-10.765108,11.460000,3.312751e+09,-9.369053,...,8.050000,3.377096e+13,161.631918,6.881506e+12,32.935727,-6.160870e+11,32.935727,-6.766790e+11,10358.259448,-5.639295
39,YR2021,11057.420754,4.820563,3.423629,6.178060,12390.808688,9.223956,10.902000,6.800104e+09,15.153015,...,5.464000,9.779907e+12,67.388391,2.474979e+12,21.902907,-8.216450e+11,21.902907,-8.613950e+11,11188.302540,1.701948
40,YR2022,11211.168317,1.323816,3.644438,5.973952,12384.109595,2.318194,10.913722,4.293535e+09,3.345875,...,5.597581,9.779907e+12,67.388391,2.527105e+12,21.900325,-8.269464e+11,21.900325,-8.739727e+11,11887.821123,0.198813


In [53]:
wb_1981 ['time'] = wb_1981 ['time'].str.replace('YR', '')
percentage_columns_1981 = ['time']
for column in wb_1981.columns:
    if "%" in column:
        percentage_columns_1981.append(column)

wb_1981  = wb_1981[percentage_columns_1981]
wb_1981_to_predict = wb_1981[-2:]
wb_1981 = wb_1981[:-2]

In [54]:
wb_1981

,time,WLD_gdp_per_capita_constant_%_growth,WLD_consumer_price_index_%,WLD_unemployment_%_of_total_labor_force,ARG_gdp_per_capita_constant_%_growth,ARG_unemployment_%_of_total_labor_force,ARG_industry_value_added_%_growth,AUS_gdp_per_capita_constant_%_growth,AUS_consumer_price_index_%,AUS_unemployment_%_of_total_labor_force,...,GBR_unemployment_%_of_total_labor_force,GBR_government_debt_total_%_of_gdp,GBR_government_expense_%_gdp,GBR_industry_value_added_%_growth,USA_gdp_per_capita_constant_%_growth,USA_consumer_price_index_%,USA_unemployment_%_of_total_labor_force,USA_government_debt_total_%_of_gdp,USA_government_expense_%_gdp,SAU_gdp_per_capita_%_growth
0,1982,-1.381191,10.240268,5.697886,-2.307766,10.977871,-3.209381,1.614208,11.351820,6.519742,...,6.330903,103.084945,37.671549,0.554029,-2.734570,6.131427,5.913032,67.388391,22.463140,-25.614130
1,1983,0.850482,8.771147,5.697886,2.681982,10.977871,5.377688,-3.437117,10.038911,6.519742,...,6.330903,103.084945,36.881620,0.554029,3.631979,3.212435,5.913032,67.388391,22.802733,-21.161576
2,1984,2.878301,8.116398,5.697886,-0.051847,10.977871,0.674286,3.418345,3.960396,6.519742,...,6.330903,103.084945,36.819563,0.554029,6.312168,4.300535,5.913032,67.388391,21.470359,-10.171216
3,1985,1.917044,6.856812,5.697886,-6.692591,10.977871,-8.651012,3.822749,6.734694,6.519742,...,6.330903,103.084945,35.932758,0.554029,3.250656,3.545644,5.913032,67.388391,22.174802,-14.608777
4,1986,1.646270,5.822667,5.697886,4.484327,10.977871,8.630264,2.339809,9.050351,6.519742,...,6.330903,103.084945,34.977858,0.554029,2.510886,1.898048,5.913032,67.388391,22.230612,11.285443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2017,2.200971,2.192010,5.557650,1.757648,8.350000,2.302833,0.587161,1.948647,5.590000,...,4.330000,159.196774,36.800352,1.258502,1.610808,2.130110,4.360000,97.763473,22.312344,-2.713853
36,2018,2.144471,2.438737,5.389680,-3.601610,9.220000,-3.049315,1.302841,1.911401,5.300000,...,4.000000,155.535877,37.101933,1.769630,2.378479,2.442583,3.900000,99.178089,22.336230,0.684349
37,2019,1.531947,2.186902,5.356915,-2.994388,9.840000,-4.790141,0.571103,1.610768,5.160000,...,3.740000,157.742820,36.322360,2.443486,1.824124,1.812210,3.670000,100.955077,22.741536,-1.322726
38,2020,-4.269491,1.920968,6.573234,-10.765108,11.460000,-9.369053,-1.278526,0.846906,6.460000,...,4.472000,190.772519,47.207681,-10.824565,-4.328618,1.233584,8.050000,161.631918,32.935727,-5.639295


In [55]:
wb_1981_to_predict

,time,WLD_gdp_per_capita_constant_%_growth,WLD_consumer_price_index_%,WLD_unemployment_%_of_total_labor_force,ARG_gdp_per_capita_constant_%_growth,ARG_unemployment_%_of_total_labor_force,ARG_industry_value_added_%_growth,AUS_gdp_per_capita_constant_%_growth,AUS_consumer_price_index_%,AUS_unemployment_%_of_total_labor_force,...,GBR_unemployment_%_of_total_labor_force,GBR_government_debt_total_%_of_gdp,GBR_government_expense_%_gdp,GBR_industry_value_added_%_growth,USA_gdp_per_capita_constant_%_growth,USA_consumer_price_index_%,USA_unemployment_%_of_total_labor_force,USA_government_debt_total_%_of_gdp,USA_government_expense_%_gdp,SAU_gdp_per_capita_%_growth
40,2022,1.323816,3.644438,5.973952,2.318194,10.913722,3.345875,1.537857,2.906272,5.224968,...,4.629382,103.084945,37.152827,-0.100020,2.114793,3.481690,5.597581,67.388391,21.900325,0.198813
41,2023,1.675105,3.815580,5.859538,1.166461,10.923384,1.851081,1.584966,2.934966,5.326272,...,4.724405,103.084945,37.142180,0.600185,1.842303,2.993147,5.689078,67.388391,21.899063,-0.161078


In [56]:
# Label the years

pre_crisis_years = ['1974', '1981', '1990', '2008', '2019']
crisis_years = ['1975', '1982', '1991', '2009', '2020']
recovery_years = ['1976', '1977', '1983', '1984', '1992', '1993', '2010', '2011']
downturn_years = ['1958', '1998', '2001', '2012']

labels = []

for i in range(len(wb_1981)):
    
    if wb_1981['time'][i] in pre_crisis_years:
        labels.append('pre_recession')
        
    elif wb_1981['time'][i] in crisis_years:
        labels.append('recession')

    elif wb_1981['time'][i] in recovery_years:
        labels.append('recovery')

    elif wb_1981['time'][i] in downturn_years:
        labels.append('downturn')
        
    else:
        labels.append('expansion')

wb_1981['year_label']= labels
wb_1981

,time,WLD_gdp_per_capita_constant_%_growth,WLD_consumer_price_index_%,WLD_unemployment_%_of_total_labor_force,ARG_gdp_per_capita_constant_%_growth,ARG_unemployment_%_of_total_labor_force,ARG_industry_value_added_%_growth,AUS_gdp_per_capita_constant_%_growth,AUS_consumer_price_index_%,AUS_unemployment_%_of_total_labor_force,...,GBR_government_debt_total_%_of_gdp,GBR_government_expense_%_gdp,GBR_industry_value_added_%_growth,USA_gdp_per_capita_constant_%_growth,USA_consumer_price_index_%,USA_unemployment_%_of_total_labor_force,USA_government_debt_total_%_of_gdp,USA_government_expense_%_gdp,SAU_gdp_per_capita_%_growth,year_label
0,1982,-1.381191,10.240268,5.697886,-2.307766,10.977871,-3.209381,1.614208,11.351820,6.519742,...,103.084945,37.671549,0.554029,-2.734570,6.131427,5.913032,67.388391,22.463140,-25.614130,recession
1,1983,0.850482,8.771147,5.697886,2.681982,10.977871,5.377688,-3.437117,10.038911,6.519742,...,103.084945,36.881620,0.554029,3.631979,3.212435,5.913032,67.388391,22.802733,-21.161576,recovery
2,1984,2.878301,8.116398,5.697886,-0.051847,10.977871,0.674286,3.418345,3.960396,6.519742,...,103.084945,36.819563,0.554029,6.312168,4.300535,5.913032,67.388391,21.470359,-10.171216,recovery
3,1985,1.917044,6.856812,5.697886,-6.692591,10.977871,-8.651012,3.822749,6.734694,6.519742,...,103.084945,35.932758,0.554029,3.250656,3.545644,5.913032,67.388391,22.174802,-14.608777,expansion
4,1986,1.646270,5.822667,5.697886,4.484327,10.977871,8.630264,2.339809,9.050351,6.519742,...,103.084945,34.977858,0.554029,2.510886,1.898048,5.913032,67.388391,22.230612,11.285443,expansion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2017,2.200971,2.192010,5.557650,1.757648,8.350000,2.302833,0.587161,1.948647,5.590000,...,159.196774,36.800352,1.258502,1.610808,2.130110,4.360000,97.763473,22.312344,-2.713853,expansion
36,2018,2.144471,2.438737,5.389680,-3.601610,9.220000,-3.049315,1.302841,1.911401,5.300000,...,155.535877,37.101933,1.769630,2.378479,2.442583,3.900000,99.178089,22.336230,0.684349,expansion
37,2019,1.531947,2.186902,5.356915,-2.994388,9.840000,-4.790141,0.571103,1.610768,5.160000,...,157.742820,36.322360,2.443486,1.824124,1.812210,3.670000,100.955077,22.741536,-1.322726,pre_recession
38,2020,-4.269491,1.920968,6.573234,-10.765108,11.460000,-9.369053,-1.278526,0.846906,6.460000,...,190.772519,47.207681,-10.824565,-4.328618,1.233584,8.050000,161.631918,32.935727,-5.639295,recession


In [57]:
category_expansion = wb_1981[wb_1981['year_label']=='expansion']
category_pre_recession = wb_1981[wb_1981['year_label']=='pre_recession']
category_recession = wb_1981[wb_1981['year_label']=='recession']
category_recovery = wb_1981[wb_1981['year_label']=='recovery']
category_downturn = wb_1981[wb_1981['year_label']=='downturn']

category_pre_recession_oversampled = resample(category_pre_recession,replace=True, n_samples = len(category_expansion))
category_recession_oversampled = resample(category_recession,replace=True, n_samples = len(category_expansion))
category_recovery_oversampled = resample(category_recovery,replace=True, n_samples = len(category_expansion))
category_downturn_oversampled = resample(category_downturn,replace=True, n_samples = len(category_expansion))

wb_1981_upsampled = pd.concat([category_expansion,
                               category_pre_recession_oversampled,
                              category_recession_oversampled,
                              category_recovery_oversampled,
                              category_downturn_oversampled], axis=0)

X = wb_1981_upsampled.drop(['time', 'year_label'],axis=1)

Y = wb_1981_upsampled['year_label']

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = pd.DataFrame(encoder.transform(Y))
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.35, shuffle=None)

scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X.shape)

(120, 88)


In [ ]:
# 1: Expansion, 2: pre_recession , 3:recession, 4:recovery , 0: downturn

In [59]:
# Build the model.
model = Sequential([ # as far as we know, all networks are sequential
    Dense(128, activation='sigmoid', input_shape=(88,)),
    Dense(64, activation='sigmoid'),      # choosing relu instead of sigmoid, this is somewhat common
    Dense(32, activation='sigmoid'),
    Dense(16, activation='sigmoid'),
    Dense(5, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  to_categorical(y_train), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=50,
  batch_size=6, # send 32 images at a time before you tweak the network again, to make it faster
)
 
    # Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

Epoch 1/50
13/13 [==============================] - 1s 2ms/step - loss: 1.6344 - accuracy: 0.2051
Epoch 2/50
13/13 [==============================] - 0s 2ms/step - loss: 1.6071 - accuracy: 0.2051
Epoch 3/50
13/13 [==============================] - 0s 2ms/step - loss: 1.5874 - accuracy: 0.2051
Epoch 4/50
13/13 [==============================] - 0s 2ms/step - loss: 1.5722 - accuracy: 0.2821
Epoch 5/50
13/13 [==============================] - 0s 2ms/step - loss: 1.5583 - accuracy: 0.3333
Epoch 6/50
13/13 [==============================] - 0s 2ms/step - loss: 1.5416 - accuracy: 0.4359
Epoch 7/50
13/13 [==============================] - 0s 2ms/step - loss: 1.5114 - accuracy: 0.4615
Epoch 8/50
13/13 [==============================] - 0s 2ms/step - loss: 1.4728 - accuracy: 0.4615
Epoch 9/50
13/13 [==============================] - 0s 2ms/step - loss: 1.4318 - accuracy: 0.4615
Epoch 10/50
13/13 [==============================] - 0s 2ms/step - loss: 1.3877 - accuracy: 0.4615
Epoch 11/50
13/13 [

[0.6057529449462891, 0.8809523582458496]

In [60]:
predictions = model.predict(X_test)
print(np.argmax(predictions, axis=1))

2/2 [==============================] - 0s 3ms/step
[0 3 4 1 3 2 1 1 2 3 2 3 3 1 2 0 2 1 3 0 1 4 3 2 1 4 0 1 2 1 2 2 4 0 1 2 0
 4 2 4 4 1]


In [61]:
y_test

,0
98,0
64,3
95,4
13,1
50,3
...,...
87,4
27,2
80,4
72,4


In [62]:
# Also here the accuracy is very good!


# This model has a much better accuracy! Will be used for predictions
model_1981_upsampled = model 
scaler_1981_upsampled  = scaler

filename = 'model_1981_upsampled.sav'
pickle.dump(model_1981_upsampled, open(filename, 'wb'))

filename= 'scaler_1981_upsampled.sav'
pickle.dump(scaler_1981_upsampled, open(filename, 'wb'))


INFO:tensorflow:Assets written to: C:\Users\luana\AppData\Local\Temp\tmpvmw5wlz0\assets


In [63]:
X = wb_1981_to_predict.drop(['time'], axis=1)
X

,WLD_gdp_per_capita_constant_%_growth,WLD_consumer_price_index_%,WLD_unemployment_%_of_total_labor_force,ARG_gdp_per_capita_constant_%_growth,ARG_unemployment_%_of_total_labor_force,ARG_industry_value_added_%_growth,AUS_gdp_per_capita_constant_%_growth,AUS_consumer_price_index_%,AUS_unemployment_%_of_total_labor_force,AUS_government_debt_total_%_of_gdp,...,GBR_unemployment_%_of_total_labor_force,GBR_government_debt_total_%_of_gdp,GBR_government_expense_%_gdp,GBR_industry_value_added_%_growth,USA_gdp_per_capita_constant_%_growth,USA_consumer_price_index_%,USA_unemployment_%_of_total_labor_force,USA_government_debt_total_%_of_gdp,USA_government_expense_%_gdp,SAU_gdp_per_capita_%_growth
40,1.323816,3.644438,5.973952,2.318194,10.913722,3.345875,1.537857,2.906272,5.224968,30.393233,...,4.629382,103.084945,37.152827,-0.100020,2.114793,3.481690,5.597581,67.388391,21.900325,0.198813
41,1.675105,3.815580,5.859538,1.166461,10.923384,1.851081,1.584966,2.934966,5.326272,30.393233,...,4.724405,103.084945,37.142180,0.600185,1.842303,2.993147,5.689078,67.388391,21.899063,-0.161078


In [64]:
# load the model from disk

filename = 'model_1981_upsampled.sav'
loaded_model = pickle.load(open(filename, 'rb'))

filename= 'scaler_1981_upsampled.sav'
loaded_scaler = pickle.load(open(filename, 'rb'))

X_scaled = loaded_scaler.transform(X)

predictions = loaded_model.predict(X_scaled)
print(np.argmax(predictions, axis=1))

1/1 [==============================] - 0s 98ms/step
[1 1]


In [ ]:
# 1: Expansion, 2: pre_recession , 3:recession, 4:recovery , 0: downturn

# Conclusion

### The first  model (accuracy 93%) predicted recession for 2021 and 2022 and expansion for 2023.

### This second model (accuracy 88%) predicted expansion in 2022 and 2023.